## **ASSIGNMENT**

##### **Heat Map**

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd

In [ ]:
# Step 1: Reading the json file to DataFrame
data = pd.read_json('/content/dutch_tweets_chunk0.json')
data.head(2)

,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,...,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",@pflegearzt @Friedelkorn @ LAguja44 Pardon wol...,2020-03-09 12:26:29,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...","I ❤️science, Therefore a Commie. ☭ FALGSC: Par...",11,0,9,3,...,"(52.5001698, 5.7480821, 0.0)",52.50017,5.748082,0.0,Flevoland,None,None,False,0.0,0.0
1,RT @grantshapps: Aviation demand is reduced du...,RT @grantshapps: Aviation demand is reduced du...,2020-03-09 12:26:34,davidiwanow,I tweet a lot but love to engage & converse. P...,I tweet a lot but love to engage and converse....,11,0,9,3,...,"(52.3727598, 4.8936041, 0.0)",52.37276,4.893604,0.0,Noord-Holland,None,None,False,0.0,0.0


In [ ]:
# Step 2: Reading necessary columns required for the heatmap

df_heatMap = data[['latitude','longitude']]
df_heatMap.head()

,latitude,longitude
0,52.50017,5.748082
1,52.37276,4.893604
2,NaN,NaN
3,NaN,NaN
4,52.37276,4.893604


In [ ]:
# Step 3: Cleaning the data
print("Cleaning data...")
df_heatMap = df_heatMap.dropna(subset=['latitude','longitude'])
df_heatMap = df_heatMap[(df_heatMap['latitude'].between(50.75,53.65))&(df_heatMap['longitude'].between(3.25,7.2))]

print(f"Total records loaded: {len(df_heatMap)}")

Cleaning data...
Total records loaded: 11745


In [ ]:
# Step 4: Create a base map centered on average coordinates
map_center = [df_heatMap['latitude'].mean(),
              df_heatMap['longitude'].mean()]
heatmap  = folium.Map(location=map_center,zoom_start=6)

In [ ]:
# Step 5: Prepare data and create heatmap
heat_data = df_heatMap[["latitude","longitude"]].values.tolist()
print("Generating heatmap...")

HeatMap(heat_data,radius=8,blur=10,max_zoom=6).add_to(heatmap)

Generating heatmap...


In [ ]:
# Step 6: Save the heatmap to an HTML file
output_file = "dutch_heatmap_new.html"
heatmap.save(output_file)
print(f"Heatmap saved as '{output_file}'")

Heatmap saved as 'dutch_heatmap_new.html'


##### **Adding Sentiment Column**

In [ ]:
from textblob import TextBlob
import pandas as pd

In [ ]:
# Step 1: Reading the json file to DataFrame
data = pd.read_json('/content/dutch_tweets_chunk0.json')
data.head(2)

,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,year,location,point_info,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern
0,"@pflegearzt @Friedelkorn @LAguja44 Pardon, wol...",@pflegearzt @Friedelkorn @ LAguja44 Pardon wol...,2020-03-09 12:26:29,TheoRettich,"I ❤️science, therefore a Commie. ☭ FALGSC: P...","I ❤️science, Therefore a Commie. ☭ FALGSC: Par...",11,0,9,3,2020,Netherlands,Nederland,"(52.5001698, 5.7480821, 0.0)",52.50017,5.748082,0.0,Flevoland,None,None,False,0.0,0.0
1,RT @grantshapps: Aviation demand is reduced du...,RT @grantshapps: Aviation demand is reduced du...,2020-03-09 12:26:34,davidiwanow,I tweet a lot but love to engage & converse. P...,I tweet a lot but love to engage and converse....,11,0,9,3,2020,"Amsterdam, The Netherlands","Amsterdam, Noord-Holland, Nederland","(52.3727598, 4.8936041, 0.0)",52.37276,4.893604,0.0,Noord-Holland,None,None,False,0.0,0.0


In [ ]:
# Step 2: Define a function to classify sentiment
def get_sentiment(text):
    analysis = TextBlob(str(text))  # Convert to string in case of NaN
    polarity = analysis.sentiment.polarity

    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
# Step 3: Apply function to create new column
data['sentiment'] = data['desc_translation'].apply(get_sentiment)

In [ ]:
# Step 4: Display result
print(data[['desc_translation', 'sentiment']].head())

                                    desc_translation sentiment
0  I ❤️science, Therefore a Commie. ☭ FALGSC: Par...  Positive
1  I tweet a lot but love to engage and converse....  Positive
2                                               None   Neutral
3                                               None   Neutral
4  Budget-Life Coach. Time management coaching. h...  Negative


In [ ]:
# Step 5: Save only 'desc_translation' and 'sentiment' columns to a CSV
data[['desc_translation', 'sentiment']].to_csv('sentiment_results.csv', index=False)

print("Sentiment results saved to 'sentiment_results.csv'")


Sentiment results saved to 'sentiment_results.csv'
